In [1]:
# -*- coding: utf-8 -*-

from matplotlib import pyplot as plt
import numpy as np
import random
from torch.utils.data import Dataset, DataLoader
import torch
import os
from torch.optim import *
from torch import nn
from torch.nn import functional as F
from scipy import ndimage
import pandas as pd
from copy import deepcopy
import random
from sklearn.metrics import f1_score
import csv
import pandas as pd
from sklearn import model_selection
import pickle

random.seed(7)
torch.cuda.manual_seed(7)
torch.manual_seed(7)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(7)

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')


In [2]:
DATA_ROOT = '/beegfs/bva212/openmic-2018'
OPENMIC = np.load(os.path.join(DATA_ROOT, 'openmic-2018.npz'))
X, Y_true, Y_mask, sample_key = OPENMIC['X'], OPENMIC['Y_true'], OPENMIC['Y_mask'], OPENMIC['sample_key']

In [3]:
train_samples = pd.read_csv(os.path.join(DATA_ROOT,'partitions/split01_train.csv'), names =['id']).to_numpy().squeeze()
test_samples = pd.read_csv(os.path.join(DATA_ROOT,'partitions/split01_test.csv'), names =['id']).to_numpy().squeeze()

In [4]:
len_data = len(train_samples)
idx_train = np.random.choice(len_data, int(len_data*0.8), replace=False)
remain_set = list(set(np.arange(len_data))-set(idx_train))

In [5]:
val_idx = np.isin(sample_key, train_samples[remain_set])
train_idx = np.isin(sample_key, train_samples[idx_train])
test_idx = np.isin(sample_key, test_samples)

In [6]:
Model_Data = []
sizes = [10, 50, 250, 500, 950]
for size in sizes:
    Example = []
    strat = model_selection.StratifiedShuffleSplit(n_splits = 1, test_size = size)
    for i in range(Y_mask.shape[1]):
        split_index = strat.split(sample_key[train_idx][Y_mask[train_idx, i]==True], (Y_true[train_idx][Y_mask[train_idx, i]==True,i]>0.5).astype(int))
        test_index = [sample_key[train_idx][Y_mask[train_idx, i]==True][test_index] for _, test_index in split_index]
        print("Size: {} Class: {}".format(size, i))
        print(test_index)
        Example.extend(test_index[0])
    Model_Data.append(Example)
Model_Data = [list(set(i)) for i in Model_Data]

Size: 10 Class: 0
[array(['115229_30720', '043532_126720', '045493_80640', '119354_19200',
       '065062_188160', '094594_111360', '057053_226560', '085406_26880',
       '138837_34560', '056529_138240'], dtype=object)]
Size: 10 Class: 1
[array(['038793_126720', '123885_145920', '061600_499200', '012802_92160',
       '031768_34560', '040995_69120', '110667_241920', '081948_99840',
       '020124_207360', '039042_65280'], dtype=object)]
Size: 10 Class: 2
[array(['124178_119040', '115702_176640', '147240_88320', '122201_165120',
       '094670_0', '151247_11520', '077140_3840', '067172_483840',
       '083660_88320', '035779_7680'], dtype=object)]
Size: 10 Class: 3
[array(['109787_42240', '066708_407040', '151958_80640', '139980_19200',
       '018295_11520', '073372_207360', '110531_61440', '107893_11520',
       '138180_26880', '133561_23040'], dtype=object)]
Size: 10 Class: 4
[array(['009170_188160', '025097_23040', '084476_744960', '021853_487680',
       '152606_403200', '091146_3

Size: 500 Class: 14
[array(['141077_299520', '138309_142080', '126063_199680', '098758_76800',
       '016047_249600', '109404_199680', '074565_7680', '060549_238080',
       '086223_257280', '085077_180480', '123885_145920', '129832_0',
       '053032_42240', '041936_7680', '116898_0', '133562_34560',
       '017021_414720', '051688_291840', '149444_122880',
       '036980_1651200', '150514_103680', '013707_19200', '051047_3840',
       '019976_23040', '139957_134400', '062145_0', '132683_107520',
       '091890_38400', '059490_72960', '012045_42240', '131527_76800',
       '019457_295680', '029777_195840', '072684_53760', '065289_30720',
       '010383_387840', '007441_299520', '071595_46080', '099350_122880',
       '057194_72960', '076657_145920', '091983_314880', '001264_407040',
       '144791_3840', '144642_257280', '065719_264960', '104432_449280',
       '141965_96000', '025703_376320', '009351_96000', '010832_760320',
       '053799_153600', '026552_53760', '009605_11520', '0

In [10]:
for i in range(len(sizes)):
    print(len(Model_Data[i]))

198
970
4236
7292
10782


In [9]:
# pickle.dump(Model_Data, open('Training_Data_Index.pkl','wb'))

In [6]:
class ArrowOfTime(Dataset):

    def __init__(self, root_dir, files, weights, label):
        self.weights = weights
        self.device = device
        self.root_dir = root_dir
        self.files = files
        self.label = label

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        logscalogram = (np.load(self.root_dir + self.files[idx]+'_cqt.npy') - -24.3633)/14.2659
        weight = self.weights[idx]
        label = (self.label[idx] > 0.5).astype(int)
        return {'logscalogram': logscalogram[np.newaxis, :], 'label': label[np.newaxis, :], 'weight': weight[np.newaxis, :]}

filenames = []
root_dir = '/beegfs/bva212/openmic-2018/cqt_full/'

BATCH_SIZE = 32

def my_collate(batch):
    data = np.stack([item['logscalogram'] for item in batch])
    target = np.concatenate([item['label'] for item in batch],axis=0)
    weight = np.concatenate([item['weight'] for item in batch],axis=0)
    weight_sum = weight.sum(0)
    weight_sum = np.repeat(weight_sum[np.newaxis, :], len(batch), 0)
    weight_sum[weight_sum==0]=1
    weight = weight/weight_sum
    return [torch.from_numpy(data).float(), torch.from_numpy(target).float(), torch.from_numpy(weight).float()]


In [7]:
class snet2_jigsaw(nn.Module):

    def __init__(self, output_classes = 20):
        '''
        Create the 5 Conv Layer Sound Net network architecture as per the paper - https://arxiv.org/pdf/1610.09001.pdf
        '''
        super(snet2_jigsaw, self).__init__()

        self.conv_layers = nn.Sequential(nn.Conv2d(in_channels = 1, out_channels= 16, kernel_size = 5, stride = 2, padding = 5), 
                                nn.BatchNorm2d(num_features = 16), 
                                nn.ReLU(inplace = True),

                                nn.Conv2d(in_channels = 16, out_channels = 32, kernel_size = 5, stride = 2, padding = 5),
                                nn.BatchNorm2d(32),
                                nn.ReLU(inplace = True),

                                nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 5, padding = 5),
                                nn.BatchNorm2d(64),
                                nn.ReLU(inplace = True),
                                nn.AvgPool2d(kernel_size = 3),

                                nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size = 4, padding = 4),
                                nn.BatchNorm2d(128),
                                nn.ReLU(inplace = True),

                                nn.Conv2d(in_channels = 128, out_channels = 256, kernel_size = 4, padding = 4),
                                nn.BatchNorm2d(256),
                                nn.ReLU(inplace = True),
                                nn.AvgPool2d(kernel_size = 3),

                                nn.Conv2d(in_channels = 256, out_channels = 512, kernel_size = 3, stride = 2, padding = 3),
                                nn.BatchNorm2d(512),
                                nn.ReLU(inplace = True),

                                nn.Conv2d(in_channels = 512, out_channels = 1024, kernel_size = 3, stride = 2, padding = 3),
                                nn.BatchNorm2d(1024),
                                nn.ReLU(inplace = True),
                                nn.AdaptiveAvgPool2d(output_size = 1)
                                )
        self.mlp_layer = nn.Linear(1024, output_classes)
              
    def forward(self, input):
        out = self.conv_layers(input)
        out = self.mlp_layer(out.view(out.shape[0], -1))
        return out
    
    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, a=0, mode='fan_in', nonlinearity='relu')
        
# Function for testing the model
def test_model(loader, model):
    correct = 0
    total_loss = 0
    total = 0
    total_num = 0
    actual_arr = []
    predicted_arr = []
    weight_array = []
    model.eval()
    with torch.no_grad():
        for spectrogram, target, weight in loader:
            spectrogram_batch, target_batch, weight_batch = spectrogram.to(device), target.to(device), weight.to(device)
            outputs = model(spectrogram_batch)
            loss = F.binary_cross_entropy_with_logits(outputs, target_batch,
                                                  weight = weight_batch,
                                                  reduction='sum')
            predicted = (torch.sigmoid(outputs.data)>0.5).float()
            
            actual_arr.extend(target.view(1,-1).squeeze().numpy().astype(int).tolist())
            predicted_arr.extend(predicted.view(1,-1).squeeze().cpu().numpy().astype(int).tolist())
            
            total_loss += loss.item()
            total += weight_batch.shape[0]
            
            correct += ((weight_batch != 0).float()*(predicted.eq(target_batch.view_as(predicted)).float())).sum().item()
            total_num += (weight_batch != 0).sum().item()
            weight_array = np.concatenate((weight_array,(weight != 0).reshape(-1).numpy().astype(int)))
        accuracy = (100 * correct / total_num)
        return accuracy, f1_score(actual_arr, predicted_arr, average='micro', sample_weight = weight_array), total_loss/total

def train_model(train_loader, val_loader, model, optimizer, scheduler, num_epochs):
    train_acc_list = []
    train_loss_list = []
    val_acc_list = []
    val_loss_list = []
    train_f1_list = []
    val_f1_list = []
    best_val_acc = 0
    for epoch in range(num_epochs):
        for spectrogram, target, weight in train_loader:
            model.train()
            spectrogram_batch, target_batch, weight_batch = spectrogram.to(device), target.to(device), weight.to(device)
            optimizer.zero_grad()
            outputs = model(spectrogram_batch)
            loss = F.binary_cross_entropy_with_logits(outputs, target_batch,
                                                  weight = weight_batch,
                                                  reduction='sum')
            loss.backward()
            optimizer.step()
        train_acc, f1_score_train, train_loss = test_model(train_loader, model)
        val_acc, f1_score_val, val_loss = test_model(val_loader, model)
        
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_state_dict = deepcopy(model.state_dict())
        
        train_acc_list.append(train_acc)
        train_f1_list.append(f1_score_train)
        train_loss_list.append(train_loss)
        val_f1_list.append(f1_score_val)
        val_acc_list.append(val_acc)
        val_loss_list.append(val_loss)
        
        scheduler.step(val_acc)
        print("Epoch:{}".format(epoch+1))
        print("Validation Accuracy:{:.2f}, Validation F1:{:.2f}, Val Loss: {:.5f}".format(val_acc, f1_score_val, val_loss))
        print("Training Acc: {:.2f}, Training F1:{:.2f}, Train Loss: {:.5f}".format(train_acc, f1_score_train, train_loss))
    return train_acc_list, train_loss_list, val_acc_list, val_loss_list, best_model_state_dict

In [8]:
import pickle
sizes = [10, 50, 250, 500, 950, 'Full']

Model_Data = pickle.load(open('Training_Data_Index.pkl', 'rb'))

Val_dataset = ArrowOfTime(root_dir, sample_key[val_idx], Y_mask[val_idx], Y_true[val_idx])
Val_loader = torch.utils.data.DataLoader(dataset = Val_dataset, 
                                              batch_size = BATCH_SIZE,
                                              shuffle = False,
                                        collate_fn = my_collate)

results_dict = {}

for i in range(len(sizes)):
    if sizes[i] != 'Full':
        sample_key_train = Model_Data[i]
        idx_train_new = np.isin(sample_key, sample_key_train)
        Y_mask_train = Y_mask[idx_train_new]
        label_train = Y_true[idx_train_new]
        Train_dataset = ArrowOfTime(root_dir, sample_key[idx_train_new], Y_mask_train, label_train)
        Train_loader = torch.utils.data.DataLoader(dataset = Train_dataset, 
                                                      batch_size = BATCH_SIZE,
                                                      shuffle = True,
                                                  collate_fn = my_collate)
    else:
        Y_mask_train = Y_mask[train_idx]
        label_train = Y_true[train_idx]
        Train_dataset = ArrowOfTime(root_dir, sample_key[train_idx], Y_mask_train, label_train)
        Train_loader = torch.utils.data.DataLoader(dataset = Train_dataset, 
                                                      batch_size = BATCH_SIZE,
                                                      shuffle = True,
                                                  collate_fn = my_collate)

    
    weights = torch.load('/beegfs/bva212/capstone/new_model/checkpoint_model.pth')['modelStateDict']
    req_weights = {key[7:]:val for key, val in weights.items()}
    model = snet2_jigsaw(2)
    model.load_state_dict(req_weights)
    model.mlp_layer = nn.Linear(1024, 20)
    model.to(device)
    learning_rate = 10**(-2)
    num_epochs = 50 # number epoch to train

    optimizer = torch.optim.Adam([param for param in model.parameters() if param.requires_grad == True], lr=learning_rate, weight_decay = 0.01)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=2, verbose=True, threshold=0.03,
                                                           threshold_mode='abs', cooldown=0, min_lr=0, eps=1e-08)

    train_acc_list, train_loss_list, val_acc_list, val_loss_list, best_model_state_dict = train_model(Train_loader, Val_loader, model, optimizer, scheduler, num_epochs)
    results_dict[sizes[i]] = {
        'train_acc_list':train_acc_list,
        'train_loss_list': train_loss_list,
        'val_acc_list': val_acc_list,
        'val_loss_list': val_loss_list,
        'best_model_state_dict': best_model_state_dict
    }
pickle.dump(results_dict, open('/scratch/rc3620/Capstone/Size_Exp_Flip_Wt_Results.pkl','wb'))

Epoch:1
Validation Accuracy:60.28, Validation F1:0.60, Val Loss: 22.98776
Training Acc: 62.69, Training F1:0.63, Train Loss: 21.47043
Epoch:2
Validation Accuracy:62.86, Validation F1:0.63, Val Loss: 0.58126
Training Acc: 67.80, Training F1:0.68, Train Loss: 0.50768
Epoch:3
Validation Accuracy:63.40, Validation F1:0.63, Val Loss: 0.37569
Training Acc: 66.67, Training F1:0.67, Train Loss: 0.39363
Epoch:4
Validation Accuracy:61.99, Validation F1:0.62, Val Loss: 0.39003
Training Acc: 64.96, Training F1:0.65, Train Loss: 0.40633
Epoch:5
Validation Accuracy:62.57, Validation F1:0.63, Val Loss: 0.38779
Training Acc: 66.67, Training F1:0.67, Train Loss: 0.41520
Epoch     5: reducing learning rate of group 0 to 1.0000e-03.
Epoch:6
Validation Accuracy:59.49, Validation F1:0.59, Val Loss: 0.39706
Training Acc: 60.98, Training F1:0.61, Train Loss: 0.43705
Epoch:7
Validation Accuracy:62.91, Validation F1:0.63, Val Loss: 0.38482
Training Acc: 67.23, Training F1:0.67, Train Loss: 0.38383
Epoch:8
Vali

Epoch:10
Validation Accuracy:68.96, Validation F1:0.69, Val Loss: 0.33944
Training Acc: 72.13, Training F1:0.72, Train Loss: 0.33654
Epoch:11
Validation Accuracy:69.62, Validation F1:0.70, Val Loss: 0.33571
Training Acc: 73.06, Training F1:0.73, Train Loss: 0.33189
Epoch:12
Validation Accuracy:69.33, Validation F1:0.69, Val Loss: 0.33563
Training Acc: 73.53, Training F1:0.74, Train Loss: 0.33296
Epoch:13
Validation Accuracy:69.83, Validation F1:0.70, Val Loss: 0.33051
Training Acc: 74.00, Training F1:0.74, Train Loss: 0.31895
Epoch:14
Validation Accuracy:69.69, Validation F1:0.70, Val Loss: 0.33113
Training Acc: 73.45, Training F1:0.73, Train Loss: 0.31192
Epoch:15
Validation Accuracy:69.74, Validation F1:0.70, Val Loss: 0.32889
Training Acc: 73.49, Training F1:0.73, Train Loss: 0.31273
Epoch:16
Validation Accuracy:70.38, Validation F1:0.70, Val Loss: 0.32835
Training Acc: 73.76, Training F1:0.74, Train Loss: 0.31232
Epoch:17
Validation Accuracy:69.83, Validation F1:0.70, Val Loss: 0.3

Epoch    18: reducing learning rate of group 0 to 1.0000e-05.
Epoch:19
Validation Accuracy:72.72, Validation F1:0.73, Val Loss: 0.29572
Training Acc: 75.49, Training F1:0.75, Train Loss: 0.28476
Epoch:20
Validation Accuracy:72.71, Validation F1:0.73, Val Loss: 0.29524
Training Acc: 75.27, Training F1:0.75, Train Loss: 0.28415
Epoch:21
Validation Accuracy:72.74, Validation F1:0.73, Val Loss: 0.29526
Training Acc: 75.30, Training F1:0.75, Train Loss: 0.28554
Epoch:22
Validation Accuracy:72.82, Validation F1:0.73, Val Loss: 0.29521
Training Acc: 75.40, Training F1:0.75, Train Loss: 0.28467
Epoch:23
Validation Accuracy:72.64, Validation F1:0.73, Val Loss: 0.29507
Training Acc: 75.23, Training F1:0.75, Train Loss: 0.28534
Epoch:24
Validation Accuracy:72.75, Validation F1:0.73, Val Loss: 0.29531
Training Acc: 75.43, Training F1:0.75, Train Loss: 0.28292
Epoch    24: reducing learning rate of group 0 to 1.0000e-06.
Epoch:25
Validation Accuracy:72.74, Validation F1:0.73, Val Loss: 0.29517
Trai

Epoch:29
Validation Accuracy:73.69, Validation F1:0.74, Val Loss: 0.28496
Training Acc: 75.99, Training F1:0.76, Train Loss: 0.27640
Epoch:30
Validation Accuracy:73.87, Validation F1:0.74, Val Loss: 0.28556
Training Acc: 76.15, Training F1:0.76, Train Loss: 0.27607
Epoch:31
Validation Accuracy:73.80, Validation F1:0.74, Val Loss: 0.28499
Training Acc: 76.09, Training F1:0.76, Train Loss: 0.27450
Epoch:32
Validation Accuracy:74.06, Validation F1:0.74, Val Loss: 0.28478
Training Acc: 76.12, Training F1:0.76, Train Loss: 0.27573
Epoch:33
Validation Accuracy:74.04, Validation F1:0.74, Val Loss: 0.28488
Training Acc: 76.18, Training F1:0.76, Train Loss: 0.27535
Epoch:34
Validation Accuracy:74.06, Validation F1:0.74, Val Loss: 0.28445
Training Acc: 76.18, Training F1:0.76, Train Loss: 0.27284
Epoch    34: reducing learning rate of group 0 to 1.0000e-05.
Epoch:35
Validation Accuracy:73.85, Validation F1:0.74, Val Loss: 0.28511
Training Acc: 76.33, Training F1:0.76, Train Loss: 0.27497
Epoch:3

Epoch:38
Validation Accuracy:73.33, Validation F1:0.73, Val Loss: 0.29334
Training Acc: 73.99, Training F1:0.74, Train Loss: 0.29523
Epoch:39
Validation Accuracy:73.16, Validation F1:0.73, Val Loss: 0.29383
Training Acc: 74.11, Training F1:0.74, Train Loss: 0.29748
Epoch    39: reducing learning rate of group 0 to 1.0000e-07.
Epoch:40
Validation Accuracy:73.12, Validation F1:0.73, Val Loss: 0.29383
Training Acc: 74.08, Training F1:0.74, Train Loss: 0.29669
Epoch:41
Validation Accuracy:73.25, Validation F1:0.73, Val Loss: 0.29326
Training Acc: 74.08, Training F1:0.74, Train Loss: 0.29645
Epoch:42
Validation Accuracy:73.04, Validation F1:0.73, Val Loss: 0.29325
Training Acc: 74.06, Training F1:0.74, Train Loss: 0.29522
Epoch    42: reducing learning rate of group 0 to 1.0000e-08.
Epoch:43
Validation Accuracy:73.30, Validation F1:0.73, Val Loss: 0.29355
Training Acc: 74.08, Training F1:0.74, Train Loss: 0.29702
Epoch:44
Validation Accuracy:73.29, Validation F1:0.73, Val Loss: 0.29357
Trai

Epoch:46
Validation Accuracy:73.19, Validation F1:0.73, Val Loss: 0.29879
Training Acc: 73.49, Training F1:0.73, Train Loss: 0.30059
Epoch:47
Validation Accuracy:73.06, Validation F1:0.73, Val Loss: 0.29825
Training Acc: 73.46, Training F1:0.73, Train Loss: 0.29991
Epoch:48
Validation Accuracy:72.93, Validation F1:0.73, Val Loss: 0.29978
Training Acc: 73.52, Training F1:0.74, Train Loss: 0.29801
Epoch:49
Validation Accuracy:73.21, Validation F1:0.73, Val Loss: 0.29841
Training Acc: 73.55, Training F1:0.74, Train Loss: 0.29972
Epoch:50
Validation Accuracy:73.17, Validation F1:0.73, Val Loss: 0.29849
Training Acc: 73.54, Training F1:0.74, Train Loss: 0.29980


In [9]:
results_dict = {}
model1 = snet2_jigsaw(20).to(device)
model = snet2_jigsaw(20).to(device)

for i in range(len(sizes)):
    if sizes[i] != 'Full':
        sample_key_train = Model_Data[i]
        idx_train_new = np.isin(sample_key, sample_key_train)
        Y_mask_train = Y_mask[idx_train_new]
        label_train = Y_true[idx_train_new]
        Train_dataset = ArrowOfTime(root_dir, sample_key[idx_train_new], Y_mask_train, label_train)
        Train_loader = torch.utils.data.DataLoader(dataset = Train_dataset, 
                                                      batch_size = BATCH_SIZE,
                                                      shuffle = True,
                                                  collate_fn = my_collate)
    else:
        Y_mask_train = Y_mask[train_idx]
        label_train = Y_true[train_idx]
        Train_dataset = ArrowOfTime(root_dir, sample_key[train_idx], Y_mask_train, label_train)
        Train_loader = torch.utils.data.DataLoader(dataset = Train_dataset, 
                                                      batch_size = BATCH_SIZE,
                                                      shuffle = True,
                                                  collate_fn = my_collate)

    
    model.load_state_dict(model1.state_dict())
    learning_rate = 10**(-5)
    num_epochs = 50 # number epoch to train

    optimizer = torch.optim.Adam([param for param in model.parameters() if param.requires_grad == True], lr=learning_rate, weight_decay = 0.01)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=2, verbose=True, threshold=0.03,
                                                           threshold_mode='abs', cooldown=0, min_lr=0, eps=1e-08)

    train_acc_list, train_loss_list, val_acc_list, val_loss_list, best_model_state_dict = train_model(Train_loader, Val_loader, model, optimizer, scheduler, num_epochs)
    results_dict[sizes[i]] = {
        'train_acc_list':train_acc_list,
        'train_loss_list': train_loss_list,
        'val_acc_list': val_acc_list,
        'val_loss_list': val_loss_list,
        'best_model_state_dict': best_model_state_dict
    }
pickle.dump(results_dict, open('/scratch/rc3620/Capstone/Size_Exp_Random_Wt_NoLinear_Results_size.pkl','wb'))

Epoch:1
Validation Accuracy:56.44, Validation F1:0.56, Val Loss: 0.41235
Training Acc: 58.52, Training F1:0.59, Train Loss: 0.43655
Epoch:2
Validation Accuracy:54.82, Validation F1:0.55, Val Loss: 0.41133
Training Acc: 58.33, Training F1:0.58, Train Loss: 0.45266
Epoch:3
Validation Accuracy:58.91, Validation F1:0.59, Val Loss: 0.40944
Training Acc: 62.50, Training F1:0.62, Train Loss: 0.45329
Epoch:4
Validation Accuracy:60.57, Validation F1:0.61, Val Loss: 0.40594
Training Acc: 65.91, Training F1:0.66, Train Loss: 0.44778
Epoch:5
Validation Accuracy:63.16, Validation F1:0.63, Val Loss: 0.39969
Training Acc: 71.40, Training F1:0.71, Train Loss: 0.43573
Epoch:6
Validation Accuracy:65.32, Validation F1:0.65, Val Loss: 0.39256
Training Acc: 76.89, Training F1:0.77, Train Loss: 0.40560
Epoch:7
Validation Accuracy:65.14, Validation F1:0.65, Val Loss: 0.38631
Training Acc: 78.03, Training F1:0.78, Train Loss: 0.39654
Epoch:8
Validation Accuracy:64.95, Validation F1:0.65, Val Loss: 0.38195
Tra

Epoch:12
Validation Accuracy:71.24, Validation F1:0.71, Val Loss: 0.32614
Training Acc: 79.49, Training F1:0.79, Train Loss: 0.29931
Epoch:13
Validation Accuracy:71.43, Validation F1:0.71, Val Loss: 0.32404
Training Acc: 79.21, Training F1:0.79, Train Loss: 0.28853
Epoch:14
Validation Accuracy:71.72, Validation F1:0.72, Val Loss: 0.32285
Training Acc: 79.68, Training F1:0.80, Train Loss: 0.28933
Epoch:15
Validation Accuracy:71.61, Validation F1:0.72, Val Loss: 0.32094
Training Acc: 80.65, Training F1:0.81, Train Loss: 0.28569
Epoch:16
Validation Accuracy:71.90, Validation F1:0.72, Val Loss: 0.32004
Training Acc: 81.12, Training F1:0.81, Train Loss: 0.27755
Epoch:17
Validation Accuracy:71.96, Validation F1:0.72, Val Loss: 0.31866
Training Acc: 82.17, Training F1:0.82, Train Loss: 0.27620
Epoch:18
Validation Accuracy:72.25, Validation F1:0.72, Val Loss: 0.31485
Training Acc: 82.06, Training F1:0.82, Train Loss: 0.26661
Epoch:19
Validation Accuracy:71.90, Validation F1:0.72, Val Loss: 0.3

Epoch:23
Validation Accuracy:76.96, Validation F1:0.77, Val Loss: 0.26954
Training Acc: 88.48, Training F1:0.88, Train Loss: 0.19653
Epoch:24
Validation Accuracy:76.98, Validation F1:0.77, Val Loss: 0.26925
Training Acc: 88.50, Training F1:0.89, Train Loss: 0.19467
Epoch:25
Validation Accuracy:77.22, Validation F1:0.77, Val Loss: 0.26932
Training Acc: 88.86, Training F1:0.89, Train Loss: 0.19465
Epoch:26
Validation Accuracy:77.30, Validation F1:0.77, Val Loss: 0.26870
Training Acc: 88.68, Training F1:0.89, Train Loss: 0.19297
Epoch:27
Validation Accuracy:77.21, Validation F1:0.77, Val Loss: 0.26780
Training Acc: 88.61, Training F1:0.89, Train Loss: 0.18677
Epoch:28
Validation Accuracy:77.35, Validation F1:0.77, Val Loss: 0.26870
Training Acc: 88.82, Training F1:0.89, Train Loss: 0.19015
Epoch:29
Validation Accuracy:77.30, Validation F1:0.77, Val Loss: 0.26814
Training Acc: 88.73, Training F1:0.89, Train Loss: 0.18696
Epoch:30
Validation Accuracy:77.25, Validation F1:0.77, Val Loss: 0.2

Epoch:33
Validation Accuracy:77.35, Validation F1:0.77, Val Loss: 0.26449
Training Acc: 88.84, Training F1:0.89, Train Loss: 0.18291
Epoch:34
Validation Accuracy:77.32, Validation F1:0.77, Val Loss: 0.26467
Training Acc: 88.87, Training F1:0.89, Train Loss: 0.18343
Epoch:35
Validation Accuracy:77.42, Validation F1:0.77, Val Loss: 0.26506
Training Acc: 88.84, Training F1:0.89, Train Loss: 0.18370
Epoch:36
Validation Accuracy:77.30, Validation F1:0.77, Val Loss: 0.26493
Training Acc: 88.84, Training F1:0.89, Train Loss: 0.18456
Epoch:37
Validation Accuracy:77.27, Validation F1:0.77, Val Loss: 0.26488
Training Acc: 88.80, Training F1:0.89, Train Loss: 0.18449
Epoch:38
Validation Accuracy:77.48, Validation F1:0.77, Val Loss: 0.26486
Training Acc: 88.57, Training F1:0.89, Train Loss: 0.18362
Epoch:39
Validation Accuracy:77.35, Validation F1:0.77, Val Loss: 0.26453
Training Acc: 88.77, Training F1:0.89, Train Loss: 0.18258
Epoch:40
Validation Accuracy:77.37, Validation F1:0.77, Val Loss: 0.2

Epoch:44
Validation Accuracy:77.92, Validation F1:0.78, Val Loss: 0.26016
Training Acc: 85.43, Training F1:0.85, Train Loss: 0.20668
Epoch:45
Validation Accuracy:77.88, Validation F1:0.78, Val Loss: 0.26082
Training Acc: 85.54, Training F1:0.86, Train Loss: 0.20692
Epoch:46
Validation Accuracy:77.82, Validation F1:0.78, Val Loss: 0.26079
Training Acc: 85.51, Training F1:0.86, Train Loss: 0.20627
Epoch:47
Validation Accuracy:77.82, Validation F1:0.78, Val Loss: 0.26114
Training Acc: 85.52, Training F1:0.86, Train Loss: 0.20745
Epoch:48
Validation Accuracy:77.87, Validation F1:0.78, Val Loss: 0.26068
Training Acc: 85.49, Training F1:0.85, Train Loss: 0.20587
Epoch:49
Validation Accuracy:77.98, Validation F1:0.78, Val Loss: 0.26086
Training Acc: 85.48, Training F1:0.85, Train Loss: 0.20675
Epoch:50
Validation Accuracy:77.82, Validation F1:0.78, Val Loss: 0.26039
Training Acc: 85.45, Training F1:0.85, Train Loss: 0.20753
Epoch:1
Validation Accuracy:71.67, Validation F1:0.72, Val Loss: 0.32

In [12]:
mean = 0.
std = 0.
for images, _, _ in Train_loader:
    batch_samples = images.size(0) # batch size (the last batch can have smaller size!)
    images1 = images.reshape(batch_samples, images.size(1), -1)
    mean += images1.mean(2).sum(0)
    std += images1.std(2).sum(0)
    
mean /= len(Train_loader.dataset)
std /= len(Train_loader.dataset)

In [13]:
print(mean, std)

tensor([-24.3633]) tensor([14.2659])
